![Banner](assets/images/banner-cap359.png)

## Projeto Final
<hr />

**Doconte:**

- Dr. Rafael Santos

**Discente**
- Adriano Almeida


## Descrição
Este projeto faz parte do trabalho final da disciplina **CAP-359 - Princípios e Aplicações de Mineração de Dados** que é oferecida no programa de pós graduação em computação aplicada do Instituto Nacional de Pesquisas Espaciais (INPE) e ministrada pelo professor Dr. Rafael Santos. O trabalho tem como objetivo identificar a partir de imagens do satélite geoestacionário GOES-16 condições de severidade em sistemas convectivos de mesoescala. Para isso, serão utilizadas as imagens com as temperaturas dos topos das nuvens, obtidas a partir do canal 13 do instrumento ABI. Será aplicada a abordagem de estatística descritiva percentil para definição das classes de esfriamento do topo das nuvens, e para a classsificação das imagens, será utilizada a rede neural convolucional.

## 1. Introdução
<hr />

Os sistemas convectivos de mesoescala são fenômenos meteorológicos caracterizados por um grande acumulado de nuvens com regioes de precipitação, podendo essas, serem moderadas e com distribuição uniforme (estratiforme), ou fortes e com regiões de ativiades intensas acompanhadas por raios (convectivas) [Houze and Robert 2004]. Em ciências atmosféricas, a escala define alguns aspectos importantes que são levados em consideração nos estudos da atmosfera, quanto à sua area, período de abrangência e métodos empregados na busca dos objetivos [Nunes 1998]. As definições de escala mais aceitas na literatura são as propostas por [Orlanski 1975], em que foram reduzidas à microescala, macroescala e mesoescala, sendo essa ultima, a que será abordada neste trabalho.

Na mesoescala são observados os eventos meteorológicos que possuem tempo de vida entre 1 a 7 dias, nas altitude de 2 km a 2000 km e com grande extensão horizontal e vertical [Orlanski 1975]. Esses sistemas são observados principalmente por radares e satélites meteorológicos. O satélite meteorológico GOES-16 (
*Geostationary Operational Environmental Satellite*) atualmente é um dos principais instrumento de observação de sistemas convectivos de mesoescala. O GOES-16 é um satélite geoestacionário que está sobre o equador no hemisferio ocidental, fornecendo informações a cerca das condições da atmosfera a curto prazo e de forma contínua [Chinchay 2018].

A bordo do satélite GOES-16 estão os instrumentos ABI (*Advanced Baseline Imager*) e GLM (*Geostationary Lightning Mapper*). O ABI possui 16 bandas espectrais operando em diferentes comprimentos de ondas, e podem monitorar características distintas da atmosfera e superfícies. O GLM é o instrumento que mede a radiância dos topos das nuvens a fim de identificar ocorrência de descargas elétricas. Os dados do GLM são agrupados em  três produtos: eventos, grupos e flashes. O evento é computado a partir de um pixel que excecede um determinado limiar de energia, o grupo é definido como um conjunto de eventos, e os flashes é um *cluster* feito a partir dos grupos, esses agrupamentos são criados a partir do algoritmo de clusterização LCFA (*Lightning Cluster Filter Algorithm*) [Goodman et al. 2013].

O objetivo deste trabalho é aplicar técnicas computacionais para identificação de severidade em sistemas convectivos de mesoescala a partir de dados coletados do satélite geoestacionario GOES-16. Para isso será utilizada a rede neural convolucional para a classificação da severiade das nuvens com base no seu nível de resfriamento.

## 2. Descrição dos dados
<hr />

Nesse trabalho foram utilizados os dados do canal 13 do instrumento ABI, que opera com comprimento de onda em 10.35 µm e mede a radiância emitida pela temperatura de brilho do topo das nuvens a cada 10 minutos. Através desse canal é possível identificar nuvens formadas exclusivamente por cristais de gelo, já que elas possuem temperaturas mais baixas [Chinchay 2018]. Para cada cena obtida a partir do satelite, foram extraídas imagens com a projeção de cada estado brasileiro.

Os dados foram baixados a partir da [AWS CLI](http://home.chpc.utah.edu/~u0553130/Brian_Blaylock/cgi-bin/goes16_download.cgi?source=aws&satellite=noaa-goes17&domain=M1&product=ABI-L2-CMIP&date=2019-07-05&hour=0) (*Amazon Web Service Command Line Interface*) e são referentes ao período de 4 e 5 de dezembro de 2019, data em que foram observadas ocorrências de tempestades em várias regiões do Brasil.

## 3. Metodologia
<hr />

A liguagem de programação utilizada neste trabalho foi [Python 3](https://www.python.o). Para a leitura dos dados foi utilizada a biblioteca [NetCDF4](https://www.unidata.ucar.edu/software/netcdf/) e o processamento foi feito com as bibliotecas [NumPy](https://numpy.org/) e [GDAL](https://gdal.org/). Para a definição dos modelos foram utilizadas as bibliotecas [Keras 3](https://keras.io) e [Scikit-learn 4](https://scikit-learn.o).

Os dados obtidos a partir do GOES 16 possui apenas uma cena a cada 10 minutos para os 16 canais do ABI. Sendo assim, foram aplicadas algumas transformações a fim de georeferenciar as imagens de acordo com a área de cada estado brasieleiro. Para isso, foram utilizadas algumas funções da biblioteca GDAL. Os dados foram representados em quatro classes com relação as temperaturas dos topos da nuvens. A criação das classes foi feita com base na distribuição dos valores de temperatura utilizando medidas de percentis, conforme mostrado na Figura [1](#fig1).

![Boxplot](results/boxplot.png)
<i id="fig1"></i>**Figura 1** - Distribuição dos valores de temperatura do topo das nuvens e percentisdestacados para criação das classes.

O gráfico da Figura [1](#fig1) mostra os percentis definidos para a criação das classes. O primeiro quartil representando os valores de temperaturas menos frequentes e foi utilizado para criar a primeira classe (nuvens muito frias), sendo essas, as nuvens com temperatura mais intensa, com o valores máximos de 282° K. A segunda classe (nuvens frias) foi criada com base no segundo quartil e possui valores de temperatura entre 283° K e 287° K. Para a terceira classe (nuvens quentes) foram utilizados os valores do terceiro quartil com valores entre 289° K e 291° K. Por fim, foram utilizados valores acimas de 291° K para representar a quarta classe (nuvens muito quentes), sendo essas as nuvens que provavelmente não possuem severidade devido a sua alta temperatura.

![CNN](results/cnn.png)
<i id="fig2"></i>**Figura 2** - Esquema da arquitetura da rede neural convolucional.

Após realizar o preprocessamento e a definição das classes, foi configurada a arquitetura da rede neural convolucional conforme ilustrada na Figura [2](#fig2). Todas as imagens foram redimensionadas para o tamanho de 128x128 e os valores de cada pixel normalizados entre 0 e 1 a fim de aliviar o a carga de processamento da rede neural. Foram utilizados 32 filtros com kernels de 16x16 e a função de ativação ReLU na camada de convolução, com a finalidade de identificar os núcleos mais frios das nuvens. A fim de destacar ainda mais os aspectos identificados pelos filtros da camada de convolução, foi aplicada uma camada de *Max pooling* reduzindo os mapas de características para as dimensões de 8x8. Em seguida foi adicionada a camada de *Flatten* para transformar os mapas de características em vetores unidimensionais para servirem como entrada para a rede densa.

Na rede neural densa foram utilizadas duas camadas ocultas, cada uma com 32 neurônios, e a fim de evitar o *overfitting* foi adicinado 30% de dropout em ambas. Na camada de saída foram adicionados 4 neurônios, representado a probabilidade da imagem processada pertencer a cada uma das classes. Foi utilizada nas camadas ocultas a função de ativação ReLU e na camada de saída a função *softmax*. A função de perda utilizada para avaliar o erro do modelo durante o treinamento, foi a *categorical cross entropy* e o otimizador foi o *Adam* com a taxa de aprendizagem (η) em 0.001.

Durante o treinamento do modelo foram utilizadas duas abordagens para a sua avaliação. A primeira abordagem foi a separação dos dados em conjuntos de teste (15%), treinamento (75%) e validação (10%), e ao final analisar o desempenho das classificações do modelo utilizando as metricas da matriz de confusão. A segunda abordagem para a avaliação do modelo foi aplicar a validação cruzada, separando os dados em 10 *folds*.

## 4. Resultados

## Referências